<a href="https://colab.research.google.com/github/FIJY/Lesson_01/blob/master/N_Gramms_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Подключаем Google Drive

In [2]:
# Подключаем гугл диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



# Установка и подключение библиотек

In [1]:
!pip install PyStemmer
!pip install wikipedia-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyStemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-linux_x86_64.whl size=579706 sha256=3309ca456b066a2dd1feaca15557053ef7083b75b5cc21e2677e1d50735ce163
  Stored in directory: /root/.cache/pip/wheels/45/7d/2c/a7ebb8319e01acc5306fa1f8558bf24063d6cec2c02de330c9
Successfully built PyStemmer


In [3]:
import pandas as pd
import sys, os, io
import json
import re
import itertools
import numpy as np
import requests
import multiprocessing
from joblib import Parallel, delayed


# Утилиты для обработки текста и генерации n-грамм


Функция генерации n-грамм:

In [6]:
# Содержимое Ngrams.py
def txt_list_to_grams(text_list, n):
    grams = []
    for txt in text_list:
        words = txt.split()
        for i in range(len(words) - n + 1):
            grams.append(' '.join(words[i:i + n]))
    return grams


Функции для обработки текста:


In [7]:
# Содержимое prepare_functions.py
RUSSIAN_SYMBOLS = list('АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя')

def has_more_than_x_russian_symbols(text, mincount=2):
    t = 0
    for s in RUSSIAN_SYMBOLS:
        if s in text:
            t += 1
            if t == mincount:
                return True
    return False

def remove_urls(vTEXT):
    return re.sub(r'((https|http)?:\/\/|www\.)(\w|\.|\/|\?|\=|\&|\%|-)*', '', vTEXT, flags=re.MULTILINE)

def remove_mails(text):
    return re.sub(r"\S*@\w*\.\w*(\s?|,|\.)", '', text)

def remove_bots(text):
    return re.sub(r"\@[\w\d]*", '', text)

def get_ngrams(arr, n=2):
    if len(arr) < n:
        yield []
    if len(arr) == n:
        yield arr
    for k in range(n, len(arr) + 1):
        yield arr[(k - n):k]

def get_sentences(txt):
    inds = []
    for i in range(len(txt)):
        if txt[i] == '.':
            if i + 1 == len(txt):
                inds.append(i + 1)
            elif txt[i + 1].isspace():
                if i + 2 < len(txt) and (not txt[i + 2].islower() or txt[i + 2] == '.'):
                    inds.append(i + 1)
    if len(inds) > 0:
        return [txt[i:j - 1] for i, j in itertools.zip_longest([0] + inds[:-1], inds)] + [txt[inds[-1] + 1:]]
    return [txt]

def split_by_words2(sentence, words):
    result = []
    tmp = []
    for w in sentence.split():
        if w in words:
            if len(tmp) > 0:
                result.append(' '.join(tmp))
                tmp = []
        else:
            tmp.append(w)
    if len(tmp) > 0:
        result.append(' '.join(tmp))
    return result


Альтернативный стеммер и функции для работы со словарём:

In [12]:
# Альтернативный стеммер
def simple_stem(text):
    return set(text.lower().split())

# Содержимое stemmer_rus.py с заменой стеммера
def Get_dictionary_values():
    return list(set(dictionary.keys()))

def Stem_text(text):
    return simple_stem(text)

import os
import io
import json
import requests

def get_stem_dictionary(url='https://raw.githubusercontent.com/NeuronsUII/SivilioBel_g/main/Shmykova/graph_skills.json?token=GHSAT0AAAAAACRKH54GYFBGRLR3DQDKO4OSZSPEE7A'):
    # Проверка, существует ли локальная копия файла
    local_filename = os.path.join(os.getcwd(), 'graph_skills.json')

    # Если файл не существует, загрузить его из интернета
    if not os.path.exists(local_filename):
        response = requests.get(url)
        response.raise_for_status()  # Проверка, что запрос успешен
        with io.open(local_filename, 'w', encoding='utf-8') as f:
            f.write(response.text)

    # Чтение файла из локальной копии
    with io.open(local_filename, 'r', encoding='utf-8') as f:
        r = json.load(f)

    return r

dictionary = get_stem_dictionary('https://raw.githubusercontent.com/NeuronsUII/SivilioBel_g/main/Shmykova/graph_skills.json?token=GHSAT0AAAAAACRKH54GYFBGRLR3DQDKO4OSZSPEE7A')


def get_soft_skills2(s_grams, h_grams, vocab=dictionary):
    full = set()
    for g in s_grams + h_grams:
        full = full.union(Stem_text(g))
    voc = vocab
    result = []
    result_set = set()
    for key, value in voc.items():
        st = set(key.split())
        if st.issubset(full):
            result_set = result_set.union(st)
            if type(value) == list:
                result += value
            else:
                result.append(value)
    inds = []
    for i in range(len(h_grams)):
        if Stem_text(h_grams[i]).issubset(result_set):
            inds.append(i)
    for i in inds[::-1]:
        del h_grams[i]
    return list(set(result))


Функции для работы с API Википедии и обработки результатов:

In [13]:
# Содержимое wikipedia_api.py с использованием requests
def wikipedia_search(query):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    }
    response = requests.get(url, params=params)
    try:
        data = response.json()
        return [result['title'] for result in data['query']['search']]
    except KeyError:
        print(f"KeyError: 'query' not found in response for query: {query}")
        return []
    except Exception as e:
        print(f"Error occurred: {e}")
        return []

def clean_list(arr):
    if len(arr) < 2:
        return arr
    res = sorted(list(set(arr)))
    deleted = []
    reg = r"\d+"
    for i in range(len(res) - 1):
        for j in range(i + 1, len(res)):
            if i not in deleted and j not in deleted:
                ri, rj = res[i].lower(), res[j].lower()
                if ri in re.sub(reg, '', rj).split():
                    if re.search(reg, rj):
                        deleted.append(j)
                    else:
                        deleted.append(i)
                elif rj in re.sub(reg, '', ri).split():
                    if re.search(reg, ri):
                        deleted.append(i)
                    else:
                        deleted.append(j)
    for i in sorted(deleted, reverse=True):
        del res[i]
    return res

def levenshtein_distance(string1, string2):
    distance = 0
    if len(string1) < len(string2):
        string1, string2 = string2, string1
    for i, v in itertools.zip_longest(string1, string2, fillvalue='-'):
        if i != v:
            distance += 1
    return distance

def levenshtein_distance_better(s1, s2, remove_desc=True):
    if remove_desc and '(' in s2:
        s2 = s2[:s2.index('(')].rstrip()
    tmp = [
        levenshtein_distance(s1, s2),
        levenshtein_distance(s1.lower(), s2),
        levenshtein_distance(s1.upper(), s2),
        levenshtein_distance(s1.title(), s2)
    ]
    if ' ' in s1:
        tmp.append(levenshtein_distance(s1.capitalize(), s2))
    return min(tmp)

def get_skills(grams, descriptions):
    result = []
    for g, o in zip(grams, descriptions):
        dists = [(w, levenshtein_distance_better(g, w)) for w in o]
        betters = [(w, l) for (w, l) in dists if l < 6 and l < len(g)]
        if len(betters) > 0:
            if len(betters) == 1:
                result.append(betters[0][0])
            else:
                words, counts = zip(*betters)
                counts = np.array(counts, dtype='int16')
                count_of_zero = np.sum(counts == 0)
                if count_of_zero < 2:
                    result.append(words[counts.argmin()])
                else:
                    counts = np.array([levenshtein_distance_better(g, w, False) for w in words])
                    if np.sum(counts == 0) < 2:
                        result.append(words[counts.argmin()])
                    else:
                        counts = np.array([levenshtein_distance(g, w) for w in words])
                        result.append(words[counts.argmin()])
    return clean_list(result)

def get_hard_skills(h):
    descriptions = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(wikipedia_search)(i) for i in h)
    return get_skills(h, descriptions)


Класс для создания графа и его узлов:

In [14]:
# Содержимое create_graph_dictionary.py
class Node:
    def __init__(self, name, number, withs=[], out=True):
        self.key = ' '.join(list(Stem_text(name)))
        self.name = name
        self.number = number
        self.withs = withs
        self.out = out

class Graph:
    def __init__(self, filename):
        with io.open(filename, 'r', encoding='utf-8') as f:
            lines = [line.strip() for line in f.readlines()]
        lines = filter(lambda line: len(line) > 0 and not line.startswith('#'), lines)

        self.nodes = []
        self.counter = 0

        for line in lines:
            out = line[-1] == '|'
            if out:
                line = line[:-1]

            nodes = [arr.split(',') for arr in line.split('|')]
            nodes1 = list(set((word.strip() for word in nodes[0])))

            if len(nodes) == 1:
                for word in nodes1:
                    self.nodes.append(Node(word, self.counter))
                    self.counter += 1
            else:
                nodes2 = [word.strip() for word in nodes[1]]
                for word in nodes2:
                    if not self.exist_node(word):
                        self.nodes.append(Node(word, self.counter))
                        self.counter += 1

                names = self.get_numbers_by_names(nodes2)
                for word in nodes1:
                    self.nodes.append(Node(word, self.counter, names, out))
                    self.counter += 1
        self.compile()

    def exist_node(self, name):
        for node in self.nodes:
            if node.name == name:
                return True
        return False

    def get_number_by_name(self, name):
        for i in range(len(self.nodes)):
            if self.nodes[i].name == name:
                return i
        return -1

    def get_numbers_by_names(self, names):
        return [self.get_number_by_name(name) for name in names]

    def get_names_by_numbers(self, numbers):
        return [self.nodes[i].name for i in numbers]

    def compile(self):
        def get_content(i):
            result = []
            it = self.nodes[i]
            for number in it.withs:
                result += get_content(number)
            if it.out:
                result.append(it.name)
            return result

        for i in range(len(self.nodes)):
            self.nodes[i].content = list(set(get_content(i)))


Функция для обработки текста и извлечения навыков:

In [15]:
# Содержимое detector.py
def get_content_from_text(lines_of_text, use_wiki=True, vocab=None):
    print(f"Processing text: {lines_of_text}")
    grams = txt_list_to_grams(lines_of_text, 2)  # Используем биграммы
    print(f"Generated grams: {grams}")

    h = []
    s = []

    for g in grams:
        if has_more_than_x_russian_symbols(g, 2):
            s.append(g)
        else:
            h.append(g)

    print(f"Soft skill grams: {s}")
    print(f"Hard skill grams: {h}")

    if vocab is None:
        soft_skills = get_soft_skills2(s, h)
    else:
        soft_skills = get_soft_skills2(s, h, vocab)

    print(f"Detected soft skills: {soft_skills}")

    if len(h) > 0 and use_wiki:
        hard_skills = get_hard_skills(h)
        soft_skills_lower = [word.lower() for word in soft_skills]
        h = [answer for answer in hard_skills if answer.lower() not in soft_skills_lower]
        print(f"Detected hard skills: {h}")
        return soft_skills + h

    return soft_skills



# Test



Чтение и обработка данных из CSV файла:

In [16]:
# Пусть к файлу с данными
csv_path = '/content/drive/MyDrive/Practice2/result_resumes.csv'

# Попробуем загрузить данные с различными разделителями и параметрами
try:
    data = pd.read_csv(csv_path, encoding='utf-8', delimiter=',', on_bad_lines='warn')
except Exception as e:
    print(f"Error reading CSV with delimiter ',': {e}")
    try:
        data = pd.read_csv(csv_path, encoding='utf-8', delimiter=';', on_bad_lines='warn')
    except Exception as e:
        print(f"Error reading CSV with delimiter ';': {e}")
        data = pd.read_csv(csv_path, encoding='utf-8', on_bad_lines='warn')

# Вывод имен всех столбцов
print(data.columns)

# Обработка определенной колонки (замените 'about_me' на существующий столбец)
column_name = 'about_me'  # укажите название вашей колонки

# Проверка наличия колонки
if column_name in data.columns:
    # Применение функции get_content_from_text к колонке
    data[column_name] = data[column_name].apply(lambda x: get_content_from_text(x.split(',')) if pd.notnull(x) else x)

    # Сохранение обработанных данных
    data.to_csv('/content/drive/MyDrive/Practice2/test.csv', index=False)
else:
    print(f"Столбец '{column_name}' не найден в данных")


Index(['id', 'gender', 'age', 'about_me', 'education', 'language',
       'date_of_birth', 'job_search_status', 'city_of_residence', 'moving',
       'business_trip', 'name', 'salary', 'specialization_schedule',
       'experience_all_period', 'experience', 'tags', 'link'],
      dtype='object')
Processing text: ['Имею опыт разработки в различных сферах на Python', ' знаком с различными СУБД. Мне интересны все возможные направления', ' где этот язык может применяться', ' в частности веб. Открыт к изучению новых технологий и решению сложных проблем. Я ищу должность бэкенд-разработчика (Django/FastAPI/DRF) или разработчик ботов (aiogram) на Python.\n\nЗнание основных алгоритмов и структур данных', ' паттернов проектирования.\n\nSoft skills: Умение работать в коллективе и в команде', ' аналитический склад ума', ' ответственность', ' стремление к самосовершенствованию', ' усидчивость.\n\nGithub: https://github.com/sgrigerc\ntg: @harcy']
Generated grams: ['Имею опыт', 'опыт разработки', 'ра

In [20]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul  5 14:32:45 2020

@author: qtckp
"""

import os

# Предположим, что функция get_content_from_text уже определена и подключена

# Примеры текста для обработки
examples = [
    [
        'Программист сайтов Битрикс и Битрикс 24.',
        'Партнёр Битрикс и Битрикс 24, интеграция Битрикс 24.',
        'Создание и продвижение сайтов, интернет магазина, лендинга.',
        'Дизайн, верстка, интернет маркетинг. Seo, топ 10, директ, соц сети.',
        'Так же работаю с любыми другими движками сайтов.',
        'Парсинг сайтов.',
        'Стаж более 15 лет на фрилансе.'
    ],
    [
        'Профессиональный опыт в сфере IT 10 лет.',
        'Опыт работы и поддержки информационных систем (сервисов) в крупных компаниях, предприятиях и гос. учреждениях.',
        '"KSB" "Россконгресс" "Burger King" "KFC"',
        'Высокая обучаемость.',
        'Грамотная речь, умение вести переговоры.',
        'Знание Английского языка: базовые знания.',
        'Управление персоналом, распределение задач.',
        'Умение самостоятельно проводить анализ и выявлять причины возникновения ошибок, и проблем.',
        'Коммуникабелен, стрессоустойчив. Без вредных привычек.'
    ],
    [
        'Студент Тольяттинской Академии управления',
        'В данный момент нахожусь на дистанционной форме обучения, поэтому ищу работу с графиком полной рабочей загруженности.',
        'Имею четкую профессиональную траекторию и всегда ответственно подхожу к выполнению возложенных на меня задач.'
    ],
    [
        'знаю JS, CSS, C++',
        'ms word, excel',
        'Мы ищем опытного Android-разработчика для разработки мобильной версии проекта по управлению жизненным циклом зданий. Требования: Опыт создания Android-приложений от 3-лет Хорошее знание методологии ООП Понимание принципов работы сервисно-ориентированной архитектуры Знание паттернов проектирование и умение их применять Желательно хорошее знание английского языка Большим плюсом будет наличие примеров работ Условия: Трудоустройство согласно ТК РФ Стабильная высокая «белая» заработная плата (зависит от уровня мастерства кандидата и обсуждается индивидуально при собеседовании) Гибкий график работы Возможность работы как в офисе, так и удаленно'
    ]
]

for i, lines in enumerate(examples):
    print(f"Example {i+1}:")
    print(get_content_from_text(lines))
    print()

# Чтение и обработка текста из файла my_resume.txt
with open("my_resume.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()
print("Processing my_resume.txt:")
print(get_content_from_text(lines, use_wiki=False))
print()

# Чтение и обработка текстов из директории learning/train_samples
for file_number in range(6):
    print(f"Processing file {file_number}:")
    file_path = os.path.join("learning", "train_samples", f"{file_number}.txt")
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    answer = get_content_from_text(lines)
    print(answer)
    print()


Example 1:
Processing text: ['Программист сайтов Битрикс и Битрикс 24.', 'Партнёр Битрикс и Битрикс 24, интеграция Битрикс 24.', 'Создание и продвижение сайтов, интернет магазина, лендинга.', 'Дизайн, верстка, интернет маркетинг. Seo, топ 10, директ, соц сети.', 'Так же работаю с любыми другими движками сайтов.', 'Парсинг сайтов.', 'Стаж более 15 лет на фрилансе.']
Generated grams: ['Программист сайтов', 'сайтов Битрикс', 'Битрикс и', 'и Битрикс', 'Битрикс 24.', 'Партнёр Битрикс', 'Битрикс и', 'и Битрикс', 'Битрикс 24,', '24, интеграция', 'интеграция Битрикс', 'Битрикс 24.', 'Создание и', 'и продвижение', 'продвижение сайтов,', 'сайтов, интернет', 'интернет магазина,', 'магазина, лендинга.', 'Дизайн, верстка,', 'верстка, интернет', 'интернет маркетинг.', 'маркетинг. Seo,', 'Seo, топ', 'топ 10,', '10, директ,', 'директ, соц', 'соц сети.', 'Так же', 'же работаю', 'работаю с', 'с любыми', 'любыми другими', 'другими движками', 'движками сайтов.', 'Парсинг сайтов.', 'Стаж более', 'более 15'

FileNotFoundError: [Errno 2] No such file or directory: 'my_resume.txt'